In [1]:
import pickle
from scipy.stats import poisson 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml
import html5lib
from string import ascii_uppercase as alphabet
import itertools
import random
from itertools import groupby

In [2]:
#read in the csv file 
all_tables = pd.read_html("https://en.wikipedia.org/wiki/2022%E2%80%9323_UEFA_Champions_League")

#Function to set all the values in the table to zero
def set_to_zero(row):
    row['Pld'] = 0
    row['W'] = 0
    row['D'] = 0
    row['L'] = 0
    row['GF'] = 0
    row['GA'] = 0
    row['GD'] = 0
    row['Pts'] = 0
    return row

#gets a table of each groupstage match
dict_table = {}

#loop through all_tables csv file to print out each group stages 
for letter, i in zip(alphabet, range(17, 25, 1)):
    df = all_tables[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df = df.iloc[:, 0:10]
    df = df.apply(set_to_zero, axis=1)
    dict_table[f'Group {letter}'] = df

#read in the cleaned 2010 to 2022 Champions League data
df_historical_data = pd.read_csv('Cleaned_Champions_League_Data_2010_to_2022.csv')
#read in the 2010 to 2022 Champions League data
df_data = pd.read_csv('Champions_League_Data_2010_to_2022.csv')
df_data

,Unnamed: 0,Team 1,Agg.,Team 2,1st leg,2nd leg,year
0,RO16,Roma,2–6,Shakhtar Donetsk,2–3,0–3,2010/11
1,RO16,Milan,0–1,Tottenham Hotspur,0–1,0–0,2010/11
2,RO16,Valencia,2–4,Schalke 04,1–1,1–3,2010/11
3,RO16,Internazionale,3–3 (a),Bayern Munich,0–1,3–2,2010/11
4,RO16,Lyon,1–4,Real Madrid,1–1,0–3,2010/11
...,...,...,...,...,...,...,...
175,Quarter,Villarreal,2–1,Bayern Munich,1–0,1–1,2021/22
176,Quarter,Benfica,4–6,Liverpool,1–3,3–3,2021/22
177,Semi,Manchester City,5–6,Real Madrid,4–3,1–3 (a.e.t.),2021/22
178,Semi,Liverpool,5–2,Villarreal,2–0,3–2,2021/22


In [3]:
dict_table['Group E']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Chelsea,0,0,0,0,0,0,0,0
1,2,Milan,0,0,0,0,0,0,0,0
2,3,Red Bull Salzburg,0,0,0,0,0,0,0,0
3,4,Dinamo Zagreb,0,0,0,0,0,0,0,0


In [4]:
#only take the 'Team 1', 'Team 2', 'year', and 'Agg.' columns
new_df = df_data[['Team 1', 'Team 2', 'year', 'Agg.']]
#remove the - to seperate the scores into two seperate columns
new_df[['Team1 goals', 'Team2 goals']] = new_df['Agg.'].str.extract(r'(\d+)\s*–\s*(\d+)', expand=True)
#replace the name Internazionale with the name Inter Milan
new_df['Team 1'] = new_df['Team 1'].replace({'Internazionale':'Inter Milan'})
new_df['Team 2'] = new_df['Team 2'].replace({'Internazionale':'Inter Milan'})
#clean the data
new_df['Team 1'] = new_df['Team 1'].str.strip()
new_df['Team 2'] = new_df['Team 2'].str.strip()


/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/3378677017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['Team1 goals', 'Team2 goals']] = new_df['Agg.'].str.extract(r'(\d+)\s*–\s*(\d+)', expand=True)
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/3378677017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[['Team1 goals', 'Team2 goals']] = new_df['Agg.'].str.extract(r'(\d+)\s*–\s*(\d+)', expand=True)
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/i

In [5]:
#read in the groupstage data
group_stage_history = pd.read_csv('group_stage_history_2010_to_2022.csv')
group_stage_history.drop('Unnamed: 0', axis=1, inplace=True)
group_stage_history.rename(columns={'score': 'Agg.'}, inplace=True)
df_group_stage_history = group_stage_history[['Team 1', 'Team 2', 'year', 'Agg.', 'Team1 goals', 'Team2 goals']]

In [6]:
#combine the group stage and knockout stage dataframes 
cl_history = pd.concat([new_df, df_group_stage_history])
cl_history

,Team 1,Team 2,year,Agg.,Team1 goals,Team2 goals
0,Roma,Shakhtar Donetsk,2010/11,2–6,2,6
1,Milan,Tottenham Hotspur,2010/11,0–1,0,1
2,Valencia,Schalke 04,2010/11,2–4,2,4
3,Inter Milan,Bayern Munich,2010/11,3–3 (a),3,3
4,Lyon,Real Madrid,2010/11,1–4,1,4
...,...,...,...,...,...,...
1243,Paris Saint-Germain,Benfica,2022/23,1–1,1,1
1244,Paris Saint-Germain,Maccabi Haifa,2022/23,7–2,7,2
1245,Benfica,Juventus,2022/23,4–3,4,3
1246,Juventus,Paris Saint-Germain,2022/23,1–2,1,2


In [7]:
df_team1 = cl_history[['Team 1', 'Team1 goals', 'Team2 goals']]
df_team2 = cl_history[['Team 2', 'Team1 goals', 'Team2 goals']]

#change the data types to numberic
df_team1['Team1 goals'] = pd.to_numeric(df_team1['Team1 goals'])
df_team1['Team2 goals'] = pd.to_numeric(df_team1['Team2 goals'])

df_team2['Team1 goals'] = pd.to_numeric(df_team2['Team1 goals'])
df_team2['Team2 goals'] = pd.to_numeric(df_team2['Team2 goals'])


/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/38154950.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team1['Team1 goals'] = pd.to_numeric(df_team1['Team1 goals'])
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/38154950.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team1['Team2 goals'] = pd.to_numeric(df_team1['Team2 goals'])
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/38154950.py:10: SettingWithCopyWarning: 
A value is trying to be 

In [8]:
df_team1 = df_team1.rename(columns={'Team 1': 'Team', 'Team1 goals': 'GoalsScored', 'Team2 goals': 'GoalsConceded'})
df_team2 = df_team2.rename(columns={'Team 2': 'Team', 'Team1 goals': 'GoalsConceded', 'Team2 goals': 'GoalsScored'})

In [9]:
#display all the columns
pd.options.display.max_rows = None

#groupby team and calculte the mean to see the average number of goals scored and conceded by each team
df_team_strength = pd.concat([df_team1, df_team2], ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
1899 Hoffenheim,1.833333,2.333333
AEK Athens,0.333333,2.166667
APOEL,0.600000,2.200000
Ajax,1.671875,1.562500
Anderlecht,0.750000,2.208333
Arsenal,2.122449,1.673469
Astana,0.833333,1.833333
Atalanta,1.904762,2.047619
Athletic Bilbao,0.833333,1.000000


In [10]:
#function to determine the number of points two teams are expected to recieve after playing eachother
def predict_points(home, away): 
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0, 11): #max number of goals that can be scored by home team
            for y in range(0, 11): #max number of goals that can be scored by away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return(points_home, points_away)
    else:
        return (0, 0)

In [11]:
predict_points('Barcelona', 'Real Madrid')

(1.2037020766810902, 1.638548321028958)

In [12]:
#assigning each group stage to a variable
A = dict_table['Group A']
B = dict_table['Group B']
C = dict_table['Group C']
D = dict_table['Group D']
E = dict_table['Group E']
F = dict_table['Group F']
G = dict_table['Group G']
H = dict_table['Group H']

In [13]:
#group stage function
def group_stage(letter):
    group = [letter.iloc[0,1], letter.iloc[1,1], letter.iloc[2,1], letter.iloc[3,1]]
    
    #create a list of all the combinations of games
    games = []
    for home, away in itertools.product(group, repeat=2):
      # Skip the combination if the home and away teams are the same
      if home != away:
        games.append([home, away])

    #create a dataframe with the combinations of games
    group_stage = pd.DataFrame(games, columns=['home', 'away'])
    return group_stage

letters = [A, B, C, D, E, F, G, H]

df_gs_list = []

# Loop through a range of values
for i in letters:
  # Create a dataframe with a single column and a single row
  df = pd.DataFrame(group_stage(i))
  # Append the dataframe to the list
  df_gs_list.append(df)


df_group_stage = pd.DataFrame()

#loop to build out the groupstage
for df in df_gs_list:
    df_group_stage = pd.concat([df_group_stage, df])
    df_group_stage['score'] = range(1, len(df_group_stage) + 1)
    df_group_stage['score'] = df_group_stage['score'].apply(lambda x: 'Match {}'.format(x))
    df_group_stage = df_group_stage[['home', 'score', 'away']]
    
df_group_stage

,home,score,away
0,Liverpool,Match 1,Napoli
1,Liverpool,Match 2,Ajax
2,Liverpool,Match 3,Rangers
3,Napoli,Match 4,Liverpool
4,Napoli,Match 5,Ajax
5,Napoli,Match 6,Rangers
6,Ajax,Match 7,Liverpool
7,Ajax,Match 8,Napoli
8,Ajax,Match 9,Rangers
9,Rangers,Match 10,Liverpool


In [14]:
grpA = df_group_stage.head(12)
grpA.drop([3, 6, 7, 9, 10, 11], axis=0, inplace=True)
grpA

/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/749413834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grpA.drop([3, 6, 7, 9, 10, 11], axis=0, inplace=True)


,home,score,away
0,Liverpool,Match 1,Napoli
1,Liverpool,Match 2,Ajax
2,Liverpool,Match 3,Rangers
4,Napoli,Match 5,Ajax
5,Napoli,Match 6,Rangers
8,Ajax,Match 9,Rangers


In [15]:
#loop to predict the points of the group stages
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    #select only the teams that are in their respective group to build the predicted group stage
    df_fixture_group_12 = df_group_stage[df_group_stage['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_12.iterrows():
        home, away = row['home'], row['away']
        #use the predict_points function to determine the winners of each group stage match
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)
    


In [67]:
dict_table['Group A']

,Team,Pts
0,Liverpool,14.0
1,Napoli,11.0
2,Ajax,9.0
3,Rangers,0.0


In [231]:
section1 = ['A Winner', 'B Winner', 'C Winner', 'D Winner', 'E Winner', 'F Winner', 'G Winner', 'H Winner']
section2 = ['A Runner', 'B Runner', 'C Runner', 'D Runner', 'E Runner', 'F Runner', 'G Runner', 'H Runner']

#create a dictionary to store the pairs of teams
pairs = {}

#loop through the teams in section 1
for team1 in section1:
  #get the letter of the team
  letter = team1[0]
  #select a random team from section 2 that does not contain the letter
  team2 = random.choice([t for t in section2 if letter not in t])
  #add the pair of teams to the dictionary
  pairs[team1] = team2
  #remove the team from section 2
  section2.remove(team2)

#build a dataframe of the randomized RO16 draw using section1 and section2
ro16_1st_leg = pd.DataFrame.from_dict(pairs, orient='index', columns=['team2'])

#build out the RO16 draw 1st leg
ro16_1st_leg.index.name = 'team1'
ro16_1st_leg.reset_index(inplace=True)
ro16_1st_leg.columns = ['home', 'away']
#combines the first and second leg of the RO16 draw
ro16_draw = pd.concat([ro16_1st_leg, ro16_1st_leg.assign(home=ro16_1st_leg['away'], away=ro16_1st_leg['home'])])
ro16_draw['score'] = range(1, len(ro16_draw) + 1)
ro16_draw['score'] = ro16_draw['score'].apply(lambda x: 'Match {}'.format(x))
ro16_draw = ro16_draw[['home', 'score', 'away']]
#display the dataframe
ro16_draw

,home,score,away
0,A Winner,Match 1,C Runner
1,B Winner,Match 2,A Runner
2,C Winner,Match 3,D Runner
3,D Winner,Match 4,H Runner
4,E Winner,Match 5,B Runner
5,F Winner,Match 6,E Runner
6,G Winner,Match 7,F Runner
7,H Winner,Match 8,G Runner
0,C Runner,Match 9,A Winner
1,A Runner,Match 10,B Winner


In [232]:
#loop to add the respective winners and runner-ups to the ro16_draw dataframe
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    group_runner_up = dict_table[group].loc[1, 'Team']
    #replace the Winner and Runner with the team names
    ro16_draw.replace({f'{group[6]} Winner': group_winner, 
                       f'{group[6]} Runner':group_runner_up}, inplace=True)
#add another column to be replaced with the winning team
ro16_draw['winner']='?'
ro16 = ro16_draw.head(8)
ro16

,home,score,away,winner
0,Liverpool,Match 1,Barcelona,?
1,Atlético Madrid,Match 2,Napoli,?
2,Bayern Munich,Match 3,Eintracht Frankfurt,?
3,Tottenham Hotspur,Match 4,Juventus,?
4,Chelsea,Match 5,Porto,?
5,Real Madrid,Match 6,Milan,?
6,Manchester City,Match 7,RB Leipzig,?
7,Paris Saint-Germain,Match 8,Borussia Dortmund,?


In [233]:
#function to get the winner of the knockout stages
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        #determine who is the winner of the match
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [234]:
#get the winner of the RO16
get_winner(ro16)

/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/3235514941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixture_updated.loc[index, 'winner'] = winner


,home,score,away,winner
0,Liverpool,Match 1,Barcelona,Barcelona
1,Atlético Madrid,Match 2,Napoli,Atlético Madrid
2,Bayern Munich,Match 3,Eintracht Frankfurt,Bayern Munich
3,Tottenham Hotspur,Match 4,Juventus,Juventus
4,Chelsea,Match 5,Porto,Chelsea
5,Real Madrid,Match 6,Milan,Real Madrid
6,Manchester City,Match 7,RB Leipzig,Manchester City
7,Paris Saint-Germain,Match 8,Borussia Dortmund,Paris Saint-Germain


In [235]:
quarter_teams = [f'winner of {ro16_draw.iloc[0,1]}', f'winner of {ro16_draw.iloc[1,1]}', f'winner of {ro16_draw.iloc[2,1]}', f'winner of {ro16_draw.iloc[3,1]}', f'winner of {ro16_draw.iloc[4,1]}', f'winner of {ro16_draw.iloc[5,1]}', f'winner of {ro16_draw.iloc[6,1]}', f'winner of {ro16_draw.iloc[7,1]}']

#randomize the teams to build out the quarter final matchups
random.shuffle(quarter_teams) 
pairs_qf = list(zip(*[iter(quarter_teams)]*2))
pairs_qf

#build out the quarter finals 1st leg
qf_lst_leg = pd.DataFrame(pairs_qf)
qf_lst_leg.columns = ['home', 'away']
#add the 2nd leg to the lst leg dataframe
qf_draw = pd.concat([qf_lst_leg, qf_lst_leg.assign(home=qf_lst_leg['away'], away=qf_lst_leg['home'])])
qf_draw['score'] = range(1, len(qf_draw) + 1)
qf_draw['score'] = qf_draw['score'].apply(lambda x: 'Match {}'.format(x))
#rename the column names
qf_draw = qf_draw[['home', 'score', 'away']]
quarter_finals = qf_draw.head(4)


In [236]:
#function update the 2nd df input with the winners of the 1st df input
def update_table(df_fixture_ro16, df_fixture_qf):
    for index, row in df_fixture_ro16.iterrows():
        winner = df_fixture_ro16.loc[index, 'winner']
        match = df_fixture_ro16.loc[index, 'score']
        df_fixture_qf.replace({f"winner of {match}": winner}, inplace=True)
    #add another column to be replaced with the winning team
    df_fixture_qf['winner'] = '?'
    return df_fixture_qf



In [237]:
#update the table
update_table(ro16, quarter_finals)
#find the winners of the quarter finals
get_winner(quarter_finals)

/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/719132843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixture_qf.replace({f"winner of {match}": winner}, inplace=True)
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/719132843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixture_qf['winner'] = '?'
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/3235514941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

,home,score,away,winner
0,Paris Saint-Germain,Match 1,Chelsea,Chelsea
1,Juventus,Match 2,Manchester City,Manchester City
2,Real Madrid,Match 3,Barcelona,Real Madrid
3,Bayern Munich,Match 4,Atlético Madrid,Bayern Munich


In [238]:
semi_teams = [f'winner of {qf_draw.iloc[0,1]}', f'winner of {qf_draw.iloc[1,1]}', f'winner of {qf_draw.iloc[2,1]}', f'winner of {qf_draw.iloc[3,1]}']

#randomize the teams to build out the semi final matchups
random.shuffle(semi_teams)
pairs_sf = list(zip(*[iter(semi_teams)]*2))

#build out the semi finals 1st leg
semi_1st_leg = pd.DataFrame(pairs_sf)
semi_1st_leg.columns = ['home', 'away']
#add the 2nd leg to the lst leg dataframe
semi_draw = pd.concat([semi_1st_leg, semi_1st_leg.assign(home=semi_1st_leg['away'], away=semi_1st_leg['home'])])
semi_draw['score'] = range(1, len(semi_draw) + 1)
semi_draw['score'] = semi_draw['score'].apply(lambda x: 'Match {}'.format(x))
#rename the column names
semi_draw = semi_draw[['home', 'score', 'away']]
semi_finals = semi_draw.head(2)


In [239]:
#update the table
update_table(quarter_finals, semi_finals)
#find the winner of the semi finals
get_winner(semi_finals)

/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/719132843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixture_qf.replace({f"winner of {match}": winner}, inplace=True)
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/719132843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixture_qf['winner'] = '?'
/var/folders/92/hw_yvh2s6ml3hlsjb02h6g5w0000gn/T/ipykernel_41185/3235514941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

,home,score,away,winner
0,Bayern Munich,Match 1,Chelsea,Bayern Munich
1,Manchester City,Match 2,Real Madrid,Real Madrid


In [240]:
final_teams = [[f'winner of {semi_draw.iloc[0,1]}'], [f'winner of {semi_draw.iloc[1,1]}']]

#add the two semi final winners to a dataframe
final = pd.DataFrame(final_teams[1], final_teams[0])
final.reset_index(inplace=True)
#rename the columns
final.columns = ['home', 'away']
final['score'] = 'Match 1'
final = final[['home', 'score', 'away']]


In [241]:
#update the table
update_table(semi_finals, final)
#get the winner of the finals
get_winner(final)

,home,score,away,winner
0,Bayern Munich,Match 1,Real Madrid,Bayern Munich


In [242]:
#print the winner of the Champions League
Champions_League_Winner = get_winner(final).iloc[0,3]
Champions_League_Winner

'Bayern Munich'